#### Summary
- We will be getting data from the Openstreetmap API with the use of the "overpass-api".
- Create a new mongodb database to hold our data.
- Run sample SQL query to show all the cafe that can be acessed with a wheel chair in Berlin,Germany.

In [1]:
from pymongo import MongoClient
import requests
import pandas as pd
import folium

ModuleNotFoundError: No module named 'pymongo'

### Get the data

In [ ]:
overpass_url = 'http://overpass-api.de/api/interpreter'
overpass_query = """
    [out:json];
    area["ISO3166-1"="DE"][admin_level=2];
    node[amenity = 'cafe'](area);
    out;"""

response = requests.get(overpass_url, params={'data': overpass_query})
data = response.json()

# properly structure the data
data = data['elements']
for i,datum in enumerate(data):
    for key in ['lat', 'lon', 'tags']:
        if key not in datum.keys():
            del data[i]
            continue
    for key in ['name', 'addr:city', 'wheelchair']:
        if key not in datum['tags'].keys():
            data[i]['tags'][key] = ''

In [ ]:
myclient = MongoClient('localhost', 27017)

# Create a database
mydb = myclient["mydatabase1"]

# Create a new collection
collection_name = 'collection1'
mycol = mydb[collection_name]

# Prepare data to a format usable by Mongodb
for i,datum in enumerate(data):
    new_dict = {}
    for k,v in datum.items():
        if k in ['lat', 'lon']: new_dict[k] = v
    for kk in ['name', 'addr:city', 'wheelchair']:
        if kk in datum['tags'].keys(): new_dict[kk] = datum['tags'][kk]
    data[i] = new_dict

In [ ]:
# Insert data into the collection
mycol.insert_many(data)

#### Run simple query to get all cafe accessible by wheelchair

In [ ]:
myquery = { "wheelchair": "yes" }

mydoc = mycol.find(myquery)

In [ ]:
new = []
for r,i in enumerate(mydoc):
    if i['wheelchair'] =='yes' and i['addr:city'] == 'Berlin':
        new.append(mydoc[r])

In [ ]:
df = pd.DataFrame(new).copy()

In [ ]:
import folium
m = folium.Map(location=df.iloc[50][['lat','lon']].astype('float').values.tolist(), zoom_start=12)

tooltip = "Click Here For More Info"

for row in range(len(df)):
    marker = folium.Marker(
        location=df.iloc[row][['lat','lon']].astype('float').values.tolist(),
        popup="<stong>{}</stong>".format(df.iloc[row]['name']),
        tooltip=tooltip)
    marker.add_to(m)

m